In [253]:
import pandas as pd
import numpy as np
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt
get_ipython().magic(u'matplotlib inline')

import featuretools as ft
import featuretools.variable_types as vtypes
from sklearn.model_selection import train_test_split, cross_validate

# mlp for multi-output regression
# from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold, cross_val_score

from keras.models import Sequential
from keras.layers import Dense

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import(
    classification_report, confusion_matrix, accuracy_score, mean_squared_error, mean_absolute_error
)
from sklearn.preprocessing import (
    StandardScaler, LabelBinarizer, FunctionTransformer,PolynomialFeatures, OrdinalEncoder
)
# from sklearn_pandas import DataFrameMapper, CategoricalImputer
from sklearn.pipeline import Pipeline, FeatureUnion

In [236]:
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor,RegressorChain
from sklearn.svm import LinearSVR
# from sklearn.multioutput import RegressorChain
from sklearn_pandas import DataFrameMapper

In [278]:
aa = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\AA-NACL.csv',header=0,
                    names=['temp','conductivity','velocity','nacl','aa'],dtype='float32')
df = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr.csv',dtype='float64')
df_test = pd.read_csv('D:\\desktop files\\weka\\Datasets\\Dr Wiabu\\aa-nacl-generated\\AA_NACL_NewFeatures_Corr_test.csv',dtype='float64')


In [85]:
aa_shuffled = aa.sample(frac=1)

In [86]:
X_train = aa_shuffled.drop(aa_shuffled.iloc[:,3:5],axis=1)
X_train

,temp,conductivity,velocity
38,4.0,29.139999,1465.199951
96,25.0,17.930000,1511.699951
21,4.0,0.100000,1421.900024
72,15.0,14.480000,1480.800049
78,15.0,38.939999,1501.900024
...,...,...,...
7,0.0,9.330000,1415.900024
9,0.0,9.720000,1418.599976
10,0.0,9.420000,1420.400024
92,25.0,17.940001,1508.099976


In [87]:
y_train = aa_shuffled.iloc[:,3:5]
y_train.head()

,nacl,aa
38,3.0,1.5
96,1.0,2.8
21,0.0,0.2
72,1.0,1.0
78,3.0,0.5


In [88]:
X_train_org = X_train.copy()
X_test_org = X_test.copy()
y_train_org = y_train.copy()
y_test_org = y_test.copy()

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train,test_size=0.2,random_state=42)

In [90]:
mapper = DataFrameMapper([
    (['temp'],  [SimpleImputer(strategy='mean')]),
#     (['temp'],  [SimpleImputer(strategy='mean'), OrdinalEncoder()]),
    (['conductivity'],  [SimpleImputer(strategy='mean'), StandardScaler()]),
    (['velocity'],  [SimpleImputer(strategy='mean'), StandardScaler()])
], df_out=True)

In [91]:
'''
This class performs feature engineering on the retrieved posts.  It is designed to be used in a pipeline
It takes no special input parameters beyond X (the dataframe to be transformed) and returns a modified version of X
with some additional, calculated columns.
Columns Created:

'''
class TitanicTransformer(TransformerMixin):
    def transform(self, X, **transform_params):
        X['vel-con'] = X['conductivity']/np.median(X['velocity'])
        return X
    def fit(self, X, y=None, **fit_params):
        return self

In [92]:
pipe1a = Pipeline([
    ('map', mapper),
    ('feature_gen', TitanicTransformer())
])

In [93]:
pipe1a = pipe1a.fit(X_train)
X_train = pipe1a.transform(X_train)

In [94]:
# X_test =  pipe1a.transform(X_test)

### Get Baseline Classification Accuracy Score on this dataframe before we add any new features

#### Start with DecisionTreeClassifier 
* (parameters previously tuned using GridSearchCV on this dataframe)

In [96]:
X_test =  pipe1a.transform(X_test)

In [134]:
[y_test.values]

[array([[1. , 0.5],
        [0. , 0.5],
        [0. , 2. ],
        [1. , 1.5],
        [1. , 0.2],
        [0. , 1.5],
        [1. , 2.8],
        [0. , 3. ],
        [0. , 3. ],
        [3. , 1.5],
        [1. , 2. ],
        [0. , 2.8],
        [1. , 2.8],
        [0. , 2. ],
        [1. , 1. ],
        [3. , 0.5],
        [1. , 2.8],
        [3. , 2. ],
        [3. , 2.8],
        [0. , 2. ],
        [3. , 1. ]], dtype=float32)]

In [181]:
#  from sklearn.metrics import explained_variance_score
# explained_variance_score(y_true, y_pred, multioutput='raw_values')

In [264]:
def run_regression(X_train, y_train,X_test,y_test):
    ESTIMATORS = {
        "K-nn": KNeighborsRegressor(),                          # Accept default parameters
        "Linear regression": LinearRegression(),
        "RandomForestRegressor": RandomForestRegressor(max_depth=4, random_state=2),
        "Decision Tree Regressor":DecisionTreeRegressor(max_depth=5),
        "RegressorChain Regressor":RegressorChain(DecisionTreeRegressor()),
        "MultiOutputRegressor Regressor":MultiOutputRegressor(LinearRegression()),
        "RegressorChain_Linear Regressor":RegressorChain(LinearRegression()),
        "MultiOutput_DecisionTree Regressor":MultiOutputRegressor(DecisionTreeRegressor())
    }
    # 9.1 Create an empty dictionary to collect prediction values
    y_test_predict = dict()
    y_mse = dict()
    for name, estimator in ESTIMATORS.items():     
        estimator.fit(X_train, y_train)                    # fit() with instantiated object
        y_test_predict[name] = estimator.predict(X_test)   # Make predictions and save it in dict under key: name
        y_mse[name] = mean_squared_error(y_test, estimator.predict(X_test))
        print('Predicted: %s' % y_mse[name])
    #     print('Predicted: %s' %  y_test_predict[name])
    return y_test_predict.items()
    
    #     scoring = ['precision_macro', 'recall_macro']
#     cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# #     n_scores = cross_validate(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = cross_val_score(wrapper, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
#     n_scores = np.absolute(n_scores)
#     # summarize performance
#     print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [265]:
run_regression(X_train, y_train,X_test,y_test)

Predicted: 0.6572763
Predicted: 0.7080265
Predicted: 0.3088927007769612
Predicted: 0.5272136781468516
Predicted: 0.590714274545511
Predicted: 0.7080265
Predicted: 0.708026484014447
Predicted: 0.768095231893517


dict_items([('K-nn', array([[1.       , 1.5      ],
       [1.       , 1.9      ],
       [0.       , 1.24     ],
       [1.       , 1.86     ],
       [1.       , 1.6      ],
       [1.       , 1.4399999],
       [1.       , 1.24     ],
       [1.       , 1.4399999],
       [0.8      , 1.96     ],
       [3.       , 1.86     ],
       [1.       , 1.24     ],
       [0.       , 1.24     ],
       [1.       , 1.6      ],
       [0.       , 1.76     ],
       [1.       , 1.6      ],
       [3.       , 1.54     ],
       [1.       , 1.04     ],
       [3.       , 1.76     ],
       [3.       , 1.54     ],
       [0.8      , 1.96     ],
       [3.       , 1.96     ]], dtype=float32)), ('Linear regression', array([[1.4793462 , 1.5352261 ],
       [1.0115418 , 1.4175586 ],
       [0.1071887 , 1.4378753 ],
       [0.9348966 , 1.4609118 ],
       [1.386753  , 1.566263  ],
       [0.64869857, 0.94139993],
       [1.1378682 , 1.664488  ],
       [0.66461504, 1.0161879 ],
       [0.89117   , 1.58

In [247]:
(print('Predicted K-nn: %s' % y_test_predict['K-nn']),
print('Predicted Linear: %s' % y_test_predict["Linear regression"]),
print('Predicted RandomForestRegressor: %s' % y_test_predict["RandomForestRegressor"]),
print('Predicted Decision Tree: %s' % y_test_predict["Decision Tree Regressor"]),
print('Predicted RegressorChain: %s' % y_test_predict["RegressorChain Regressor"]),
print('Predicted MultiOutputRegressor: %s' % y_test_predict["MultiOutputRegressor Regressor"]))

Predicted K-nn: [[1.        1.5      ]
 [1.        1.9      ]
 [0.        1.24     ]
 [1.        1.86     ]
 [1.        1.6      ]
 [1.        1.4399999]
 [1.        1.24     ]
 [1.        1.4399999]
 [0.8       1.96     ]
 [3.        1.86     ]
 [1.        1.24     ]
 [0.        1.24     ]
 [1.        1.6      ]
 [0.        1.76     ]
 [1.        1.6      ]
 [3.        1.54     ]
 [1.        1.04     ]
 [3.        1.76     ]
 [3.        1.54     ]
 [0.8       1.96     ]
 [3.        1.96     ]]
Predicted Linear: [[1.4793462  1.5352261 ]
 [1.0115418  1.4175586 ]
 [0.1071887  1.4378753 ]
 [0.9348966  1.4609118 ]
 [1.386753   1.566263  ]
 [0.64869857 0.94139993]
 [1.1378682  1.664488  ]
 [0.66461504 1.0161879 ]
 [0.89117    1.586407  ]
 [2.303678   1.8711286 ]
 [1.1370847  1.6324435 ]
 [0.11374581 1.4593792 ]
 [1.375064   1.6824007 ]
 [0.47450423 1.5612383 ]
 [1.4040318  1.6031348 ]
 [2.3755124  1.7386897 ]
 [1.0926173  1.1139066 ]
 [1.8937321  1.3556645 ]
 [2.3065836  1.805016  ]
 [0.882

(None, None, None, None, None, None)

#### Running regression on the extracted dataset manually removed some features

In [280]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Id                                   21 non-null     float64
 1   temp                                 21 non-null     float64
 2   conductivity = 0.6872724890708923    21 non-null     float64
 3   conductivity = -0.45598065853118896  21 non-null     float64
 4   conductivity = 1.5027929544448853    21 non-null     float64
 5   conductivity = 0.654707133769989     21 non-null     float64
 6   conductivity = 0.642235279083252     21 non-null     float64
 7   conductivity = 0.6283775568008423    21 non-null     float64
 8   conductivity = -0.42895829677581787  21 non-null     float64
 9   conductivity = -0.45736637711524963  21 non-null     float64
 10  conductivity = -1.0483936071395874   21 non-null     float64
 11  conductivity = -1.0504722595214844

In [281]:
run_regression(df, y_train,df_test,y_test)

Predicted: 1.6618667
Predicted: 1.1086955743145341
Predicted: 1.3063790952987835
Predicted: 1.3646447194308684
Predicted: 2.7640476141089487
Predicted: 1.1086955743145337
Predicted: 1.1086955743145332
Predicted: 2.8409523744384453


dict_items([('K-nn', array([[1.8       , 1.5600001 ],
       [1.8       , 2.52      ],
       [1.8       , 1.1       ],
       [1.8       , 1.1       ],
       [2.4       , 2.06      ],
       [2.4       , 0.93999994],
       [1.8       , 1.14      ],
       [2.4       , 2.42      ],
       [1.8       , 1.6       ],
       [1.4       , 0.93999994],
       [1.8       , 1.24      ],
       [0.6       , 1.96      ],
       [1.2       , 2.1       ],
       [1.        , 1.4       ],
       [1.2       , 1.6       ],
       [0.6       , 2.        ],
       [1.2       , 1.5       ],
       [1.8       , 1.4399999 ],
       [1.        , 1.74      ],
       [1.        , 1.74      ],
       [1.        , 1.74      ]], dtype=float32)), ('Linear regression', array([[1.29736166, 1.55394936],
       [1.1859945 , 1.63089948],
       [1.5602305 , 1.46399683],
       [1.34957079, 1.54675396],
       [1.43495502, 1.52490248],
       [1.44323193, 1.49771736],
       [1.69583201, 1.41392309],
       [1.25228

In [ ]:
X_train_aray = X_train.to_numpy()
X_train_aray.shape

X_test_aray = X_test.to_numpy()
X_test_aray.shape

In [ ]:
run_reg_makePred(X_train_aray, y_train,X_test_aray)

## FEATURETOOLS FEATURE EXTRACTION

In [ ]:
es = ft.EntitySet(id = 'hydrate')
es.entity_from_dataframe(entity_id = 'concentration', dataframe = X_train, index = 'Id')

In [ ]:
es["concentration"].variables

In [ ]:
es = es.normalize_entity(base_entity_id='concentration', new_entity_id='conductivity2', index='conductivity')

In [ ]:
es

*  tell DFS to ignore the PassengerId column( we don't want to generate any features for it) using the command ignore_variables={'Passengers':['PassengerId']}),

In [ ]:
feature_matrix, feature_names = ft.dfs(entityset=es, 
    target_entity = 'concentration', 
    max_depth = 2, 
    verbose = 3, 
    n_jobs = 1)

In [ ]:
Xf_train=feature_matrix.copy() # save the generated features back into our X_Train
feature_matrix.head().T

* Repeat the steps for Encoding our training set

In [ ]:
feature_matrix_enc, features_enc = ft.encode_features(feature_matrix, feature_names, include_unknown=False)
feature_matrix_enc.head()

In [ ]:
Xf_train = feature_matrix_enc.copy()
Xf_train.head()

Now create a new entity set for test and repeat the steps for adding the Passengers and PClass entities¶

In [ ]:
# creating and entity set 'es'
es_tst = ft.EntitySet(id = 'hydrate')
# adding a dataframe 
es_tst.entity_from_dataframe(entity_id = 'concentration', dataframe = X_test, index = 'Id')
# add PCLass entity
es_tst = es_tst.normalize_entity(base_entity_id='concentration', new_entity_id='conductivity2', index='conductivity')

In [ ]:
es_tst

### Now call calculate_feature_matrix on our test entity set and pass in the list of saved features from training¶

In [ ]:
feature_matrix_tst = ft.calculate_feature_matrix(features=features_enc, entityset=es_tst)

In [ ]:
Xf_test = feature_matrix_tst.copy()
Xf_test.head()

In [ ]:
Xf_test.tail(2)

In [ ]:
# double check for nulls
for col in Xf_train.columns:
    if Xf_train[col].isnull().sum() >0:
        print(col)
        Xf_train.drop(col, axis=1, inplace=True)

In [ ]:
# double check for nulls
for col in Xf_test.columns:
    if Xf_test[col].isnull().sum() >0:
        print(col)
        Xf_test.drop(col, axis=1, inplace=True)

### Now remove highly correlated features

In [ ]:
# Threshold for removing correlated variables
threshold = 0.7

# Absolute value correlation matrix
corr_matrix = Xf_train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# Select columns with correlations above threshold
collinear_features = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d features to remove.' % (len(collinear_features)))

In [ ]:
Xf_train_flt = Xf_train.drop(columns = collinear_features)
Xf_test_flt = Xf_test.drop(columns = collinear_features)
Xf_train_flt.shape, Xf_test_flt.shape
